In [7]:
from tensorflow.keras.applications import InceptionV3
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import listdir
from os.path import isfile, join
import math

In [8]:
class SequenceGenerator(keras.utils.Sequence):
    """
    A keras Sequence to be used as an image generator for the model.
    """

    def __init__(self, x, y, batchsize):
        self.x, self.y, self.batchsize = x, y, batchsize

    def __len__(self):
        return math.ceil(len(self.x) / self.batchsize)

    def __getitem__(self, idx):
        x_names = self.x[idx * self.batchsize:(idx + 1) * self.batchsize]
        y_names = np.asarray(self.y[idx * self.batchsize:(idx + 1) * self.batchsize])
        
        # open x image names, resize, normalise and make a numpy array
        batch_x = np.array([np.asarray(Image.open(file_name).resize((299, 299))) for file_name in x_names]) / 255.0

        return batch_x, y_names

    def num_classes(self):
        ret = []
        for cat in self.y:
            if cat not in ret:
                ret.append(cat)
        return len(ret)

In [9]:
def get_filenames_from_dir(directory, num_categories=1000):
    x = []
    y = []
    category_count = 0
    for category in listdir(directory):
        for file in listdir("{0}/{1}".format(directory, category)):
            x.append("{0}/{1}/{2}".format(directory, category, file))
            y.append(category_count)
        category_count += 1
        if category_count == num_categories:
            break

    return x, y

In [10]:
# makes arrays of the images and label names
x_names, y_names = get_filenames_from_dir("database", 50)

# 15% of all the images are set aside as the test set
x_train_val, x_test, y_train_val, y_test = train_test_split(x_names, y_names, test_size=0.15, random_state=42)

# 17% of the non-test images are set aside as the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.17, random_state=42)

In [11]:
# make generators with batch size 16 for each set
train_gen = SequenceGenerator(x_train, y_train, 16)
val_gen = SequenceGenerator(x_val, y_val, 16)
test_gen = SequenceGenerator(x_test, y_test, 16)

In [12]:
# Inception model
base_model = InceptionV3(weights="imagenet")
for layer in base_model.layers:
    layer.trainable = False

#predictions = keras.layers.GlobalAveragePooling2D()(base_model.output)
predictions = keras.layers.Dense(1024, activation='relu')(base_model.output)
predictions = keras.layers.Dense(2048, activation='relu')(predictions)
predictions = keras.layers.Dense(1024, activation='relu')(predictions)
predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(predictions)

model = keras.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_gen, validation_data=val_gen, epochs=5)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 605 steps, validate for 124 steps
Epoch 1/10
605/605 [==============================] - 157s 259ms/step - loss: 3.2847 - accuracy: 0.1020 - val_loss: 3.1645 - val_accuracy: 0.1079
Epoch 2/10
605/605 [==============================] - 138s 229ms/step - loss: 2.7054 - accuracy: 0.1770 - val_loss: 3.2695 - val_accuracy: 0.1326
Epoch 3/10
605/605 [==============================] - 138s 229ms/step - loss: 2.4609 - accuracy: 0.2336 - val_loss: 3.6183 - val_accuracy: 0.0888
Epoch 4/10
605/605 [==============================] - 138s 229ms/step - loss: 2.2861 - accuracy: 0.2745 - val_loss: 4.6289 - val_accuracy: 0.0726
Epoch 5/10
605/605 [==============================] - 138s 229ms/step - loss: 2.1434 - accuracy: 0.3126 - val_loss: 3.1553 - val_accuracy: 0.1508
Epoch 6/10
605/605 [==============================] - 138s 229ms/step - loss: 2.0335 - accuracy: 0.3464 - val_loss: 4.0833 - val_accuracy: 0.1114
Epoch 7/10
605/605 [==========

In [ ]:
model.evaluate(test_gen)

  ...
    to  
  ['...']
129/129 [==============================] - 24s 188ms/step - loss: 3.7637 - accuracy: 0.1463


[3.7636766489162, 0.1462585]